# Homework 3 

You will have 2 datasets to work on. 
#### You have to answer questions in this [form](https://goo.gl/forms/5gfxvKZxpoydoeOB2) and provide your code

### 1. Wikipedia Web Traffic Time Series

Data from [Kaggle competition](https://www.kaggle.com/c/web-traffic-time-series-forecasting)* )

*wikipedia_train3* - train data *wikipedia_test3* - test data created by us from original train data . For more information about dataset, please visit Homework1 assignment


### 2. French dataset

Target variable is prime_tot_ttc – house of insurance price.<p>
The variables in the files are:<p>
code_postal: postal code of the insured property<p>
nb_pieces: number of rooms<p>
veranda: presence of a veranda<p>
biens_nomades: ceiling of coverage of goods (laptop, tablets, musical instruments ,…)<p>
sit_fam: family situation of the insured<p>
profession: profession of the insured<p>
nature_lri: nature of the insured property<p>
societe: company covering the risk<p>
cdomact: area of activity of the insured<p>
niv_urbanisation: level of urbanization in which is the insured property<p>
age_societaire: age of the member<p>
objets_precieux: presence of precious objects in the insured property<p>
pres_contrat_auto: holding a contract of auto insurance<p>
formule: guarantee formula<p>
sit_juridique: legal situation<p>
surf_habitat: area of insured property<p>
patrimoine_mob: amount of heritage insured furniture<p>
var1 - ... - var11: non-explicit variables<p>


## Wikipedia page views (SMAPE metric)

In [2]:
import pandas as pd
import numpy as np
import math
from dateutil.relativedelta import relativedelta

In [91]:
### This is data from wikipedia3 archive in data folder

train = pd.read_csv("../data/wikipedia_train3.csv", parse_dates=['date'])
test = pd.read_csv("../data/wikipedia_test3.csv", parse_dates=['date'])

**1.** Take a look carefuly at train and test dataset. Note for you what is the difference in them and how they are dependent. **For _only_ train** create a holdout validation using any type of split you think is useful here. What is the split type you are using? Answer in google forms

**2.** Write a code to compare the score of your validation and test set. For scoring, use metric SMAPE (code is in lecture). For prediction use 15 previous days median. In the google form write your validation score.

**3.** Perform K-fold validation using your type of split. Run GridSearch with any classificator you like and set of parameters to optimize, providing it with your custom validation.  Compare the score of your validation and test set. For scoring, again, use metrics SMAPE. In the google form write your scores on validation and test sets

In [92]:
train.tail()

,Page,date,Visits
2882855,The_Revenant_–_Der_Rückkehrer_de.wikipedia.org...,2016-08-31,942.0
2882856,Especial:Entrar_es.wikipedia.org_all-access_sp...,2016-08-31,1193.0
2882857,Especial:Libro_es.wikipedia.org_all-access_spider,2016-08-31,1862.0
2882858,Special:Search_es.wikipedia.org_all-access_spider,2016-08-31,7217.0
2882859,Wikipedia:Portada_es.wikipedia.org_all-access_...,2016-08-31,57752.0


In [3]:
import pandas_profiling as pp
pp.ProfileReport(train)

Number of variables,3
Number of observations,2882860
Total Missing (%),0.0%
Total size in memory,66.0 MiB
Average record size in memory,24.0 B
Numeric,1
Categorical,1
Boolean,0
Date,1
Text (Unique),0
Rejected,0


In [4]:
pp.ProfileReport(test)

Number of variables,3
Number of observations,732530
Total Missing (%),0.0%
Total size in memory,16.8 MiB
Average record size in memory,24.0 B
Numeric,1
Categorical,1
Boolean,0
Date,1
Text (Unique),0
Rejected,0


## Task1

In [97]:
#Train set is 4 months and validation is the rest chronologically
train=train.fillna(0)
split_date = train.iloc[0]['date'] + relativedelta(months=4)
train_train, train_test = train[train['date'] < split_date], train[train['date'] >= split_date] 
print(train_test.shape[0] /(train_train.shape[0] + train_test.shape[0]))

0.5040983606557377


## Task2

In [98]:
def smape_fast(y_true, y_pred):
    out = 0
    for i in range(y_true.shape[0]):
        a = y_true.iloc[i]
        b = y_pred[i]
        if b < 1:
            b = 0
        c = a+b
        if c == 0:
            continue
        out += math.fabs(a - b) / c
    out *= (200.0 / y_true.shape[0])
    return out

In [99]:
def median_pred(days_num, train, cv, test):
    df_median = train.groupby('Page').apply(lambda x: x.tail(days_num).median()).reset_index()
    df_median.columns = ['Page', 'pred_Visits']
    cv = cv.merge(df_median, how='left')
    median_cv_pred = smape_fast(cv['Visits'], cv['pred_Visits'])
    test = test.merge(df_median, how='left')
    median_test_pred = smape_fast(test['Visits'], test['pred_Visits'])
    print("Median pred smape cv - {}".format(median_cv_pred))
    print("Median pred smape test - {}".format(median_test_pred))
    return median_cv_pred, median_test_pred

In [100]:
median_pred(15, train_train, train_test, test)

Median pred smape cv - 47.172439136156605
Median pred smape test - 51.77204594151946


(47.172439136156605, 51.772045941519458)

In [46]:
import re
def get_language(page):
    res = re.search('[a-z][a-z].wikipedia.org',page)
    return res.group(0)[0:2] if res else 'na'

In [40]:
def get_access_method(page):
    res = page.split('_')[-2]
    return res if res else 'na' 

In [101]:
train['Month'] = train["date"].dt.month
train['Day'] = train["date"].dt.day
train['DayOfWeek'] = train["date"].dt.dayofweek
test['Month'] = test["date"].dt.month
test['Day'] = test["date"].dt.day
test['DayOfWeek'] = test["date"].dt.dayofweek

In [102]:
def generate_features(train_df, test_df):
    train_df['Loc'] = train_df['Page'].apply(get_language)
    train_df['Agent'] = train_df['Page'].apply(get_access_method)
    test_df['Loc'] = test_df['Page'].apply(get_language)
    test_df['Agent'] = test_df['Page'].apply(get_access_method)

In [103]:
generate_features(train, test)
train.head()

,Page,date,Visits,Month,Day,DayOfWeek,Loc,Agent
0,Special:Search_zh.wikipedia.org_all-access_spider,2016-01-01,2660.0,1,1,4,zh,all-access
1,Wikipedia:首页_zh.wikipedia.org_all-access_spider,2016-01-01,9936.0,1,1,4,zh,all-access
2,Project:上传_zh.wikipedia.org_all-access_spider,2016-01-01,1712.0,1,1,4,zh,all-access
3,Special:用户登录_zh.wikipedia.org_all-access_spider,2016-01-01,3175.0,1,1,4,zh,all-access
4,Championnat_d'Europe_de_football_2016_fr.wikip...,2016-01-01,1682.0,1,1,4,fr,desktop


In [104]:
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder
def __encode_categorical(df_list, cat_cols):
    # initialize placeholder
    d = defaultdict(LabelEncoder)
    # fit and encode train/test,
    codes = pd.concat(
        [df[cat_cols] for df in df_list],
        axis=0
    ).fillna('').apply(
        lambda x: d[x.name].fit(x)
    ),
    # transform encodings to train/test etc
    for df in df_list:
        df[cat_cols] = df[cat_cols].fillna('').apply(
            lambda x: d[x.name].transform(x))

In [105]:
__encode_categorical(df_list=[train, test], cat_cols=['Loc', 'Agent'])

In [108]:
### Your code here
def create_validation(df, start_date, window):
    return df.loc[(df['date'] >= pd.to_datetime(start_date) - relativedelta(days=0)) & \
                  (df['date'] <  pd.to_datetime(start_date) + relativedelta(days=window))].index, \
           df.loc[(df['date'] >= pd.to_datetime(start_date) + relativedelta(days=window)) & \
                  (df['date'] <  pd.to_datetime(start_date) + relativedelta(days=window*2))].index

In [109]:
def create_cv_iter(train, window=10):
    ### A validation iterator
    CVIterator = []
    #make validation set of test set size
    starting_date = train.iloc[0]['date']
    while starting_date <= train.iloc[-1]['date'] - relativedelta(days=window*2):
        trainIndices, valIndices = create_validation(train, starting_date, window)
        CVIterator.append( (trainIndices, valIndices) )
        starting_date += relativedelta(days=window)
    return CVIterator
CVIterator = create_cv_iter(train)

In [110]:
X_train, y_train = train.drop(['Visits', 'date', 'Page'], axis=1), train.Visits
X_test, y_test = test.drop(['Visits', 'date', 'Page'], axis=1), test.Visits

In [123]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.pipeline import Pipeline
regressor = Ridge(random_state=42, normalize=True)

pipeline_r = Pipeline([('regressor', regressor)])
param_grid = {
    'regressor__alpha': [0.001, 0.01, 0.1, 1., 2., 5.]
}
grid_search = GridSearchCV(pipeline_r, param_grid=param_grid, cv=CVIterator,
                           scoring=make_scorer(smape_fast, greater_is_better=False))

%time grid_search.fit(X_train,y_train)

Wall time: 5min 58s


GridSearchCV(cv=[(Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                 8,      9,
            ...
            118140, 118141, 118142, 118143, 118144, 118145, 118146, 118147,
            118148, 118149],
           dtype='int64', length=118150), Int64Index([118150, 118151, ...2835595, 2835596,
            2835597, 2835598, 2835599],
           dtype='int64', length=118150))],
       error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('regressor', Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=True, random_state=42, solver='auto', tol=0.001))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'regressor__alpha': [0.001, 0.01, 0.1, 1.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(smape_fast, greater_is_better=False), verbose=0)

In [124]:
grid_search.best_params_

{'regressor__alpha': 1.0}

In [125]:
# Average SMAPE score over K folds for best parameters set
print("Validation SMAPE score", math.fabs(grid_search.best_score_))

Validation SMAPE score 105.87709773689822


In [126]:
# Test SMAPE score for best parameters set
print("Test SMAPE score", math.fabs(grid_search.score(X_test, y_test)))

Test SMAPE score 120.7700367085155


## French house insurance prediction (MAPE metric)

**1.** Take a look carefuly at train and test dataset. Note for you what is the difference in them and how they are dependent. You may use any tool you like, any EDA you want. Decribe your validation and reason for your choice in google form.

**2.** 
+ Implement your validation. 
+ Generate additional features. 
+ Build a model you like/you can. 
+ Wait for training your model. If it takes too much time, listen to https://www.youtube.com/watch?v=XWkGL4eZR3E
+ Submit your scores on validation and test to google form. Use MAPE metric for scoring. 
+ Ланфрен ланфра лантатита

In [ ]:
train = pd.read_csv('../data/train_french.csv')
test = pd.read_csv('../data/test_french.csv')